In [1]:
pip install -q peft transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 16.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0

In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.1 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 9.4 MB/s eta 0:00:00:00:01


In [4]:
!pip install sentencepiece

In [5]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
import os
import torch
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline,
logging,
HfArgumentParser,
TrainingArguments
)
from peft import PeftModel, LoraConfig
from trl import SFTTrainer
import re

2025-07-02 14:37:01.474216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751467021.662747      77 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751467021.719448      77 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [65]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamadghafghaziyan/persian-medical-qa-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/persian-medical-qa-dataset


In [7]:
dataset = load_dataset('chrishayuk/test', split = 'train')

README.md:   0%|          | 0.00/36.0 [00:00<?, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/43 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 43
})

In [66]:
os.listdir('/kaggle/input/persian-medical-qa-dataset')

['final_dataset.csv']

In [67]:
df = pd.read_csv('/kaggle/input/persian-medical-qa-dataset/final_dataset.csv')
df.head()

,question,answer
0,با سلام من49سالمه سه سال پیش یائسه شدم و الان ...,با توجه به ضخامت بالا نمونه برداري لازمه مصرف ...
1,من یک زن متاهل هستم که متاسفانه26روز پیش مورد ...,بله احتمالا متاسفانه از همون رابطه بوده چون سن...
2,سلام خوب هستین من یک هفتس دردپریودی دارم اما پ...,قرص اورژانسي دو نبايد مصرف ميکرديد اگر رابطه ي...
3,باسلام.در سونوی من نوشته شده حدود4تا5سی سی مای...,اين قرص ها ارتباطي به اون مايع ندارن و بر اساس...
4,سلام اقای دکتر\nمن حین شام خوردن یکی ازبرراکتا...,سلام به دندانپزشکتان جهت نصب مجدد براکت مراجعه...


In [68]:
df.shape

(11427, 2)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11427 entries, 0 to 11426
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  11427 non-null  object
 1   answer    11427 non-null  object
dtypes: object(2)
memory usage: 178.7+ KB


In [70]:
df.isnull().sum()

question    0
answer      0
dtype: int64

In [163]:
df['question'][0]

'با سلام من49سالمه سه سال پیش یائسه شدم و الان خونریزی دارم 5روزهست دیروز سونودادم جوابش گفت آندومتروضخامت رحم 9mmهست امروز جمعه بودم نتونستم به دکتری نشان دهم نگرانم\nواین هم هست که من 20روز بود از کاکوتی استفاده می\u200cکردم در چایی هام ودوسه به هم زعفران دم کرده خوردم\nنشانه بیماری من چی هست\nممننونم'

In [164]:
'<s>[INST] ' + df['question'][0] + ' [/INST] ' + df['answer'][0] + ' </s>'

'<s>[INST] با سلام من49سالمه سه سال پیش یائسه شدم و الان خونریزی دارم 5روزهست دیروز سونودادم جوابش گفت آندومتروضخامت رحم 9mmهست امروز جمعه بودم نتونستم به دکتری نشان دهم نگرانم\nواین هم هست که من 20روز بود از کاکوتی استفاده می\u200cکردم در چایی هام ودوسه به هم زعفران دم کرده خوردم\nنشانه بیماری من چی هست\nممننونم [/INST] با توجه به ضخامت بالا نمونه برداري لازمه مصرف دمنوش و ..رو قطع کنيد </s>'

In [178]:
df_b = df[50:101]

In [179]:
df_b.head()

,question,answer,answer_t
50,سلام. وقت خوش\nمن به ویروس گوارشی مبتلا شدم (ش...,سلام تا ساعت,سلام تا ساعت
51,با سلام...بنده 27 فروردین آخرین پریودیم بوده.....,باردار هستيد و يک هفته ديگه سونو بديد,باردار هستيد و يک هفته ديگه سونو بديد
52,باسلام\nچهل ساله هستم فرزند سوم غربالگری اول م...,امنيوسنتز يک درصد خطر سقط داره اما قطعا بايد ا...,امنيوسنتز يک درصد خطر سقط داره اما قطعا بايد ا...
53,سلام وقت بخیر من دوهفته پیش هنگام رابطه مویرگ ...,حتما معاينه و پاپ اسمير لازمه,حتما معاينه و پاپ اسمير لازمه
54,بنده 4 ماهه سزارین با بیهوشی عمومی شدم بعد زای...,سلام\nبراي مشاوره تلفني و يا مشاوره حضوري با ر...,سلام\nبراي مشاوره تلفني و يا مشاوره حضوري با ر...


In [180]:
df_b.reset_index(drop = True, inplace = True)

In [181]:
b = []
for i in range(df_b.shape[0]):
    b.append('<s>[INST] ' + df_b['question'][i] + ' [/INST] ' + df_b['answer'][i] + ' </s>')

In [183]:
df_b['answer'][10]

'سلام البته اين آزمايش ممکنه به خاطر داروها يا شرايطي هم مثبت بشه ولي با توجه به سن تون بايد کولونوسکوپي هم انجام دهيد'

In [184]:
b

['<s>[INST] سلام. وقت خوش\nمن به ویروس گوارشی مبتلا شدم (شنا میکنم). میخواستم بدونم از شروع علائم مثل تهوع و دلدرد تا چند روز نمیتونم از استخر استفاده کنم و این تا چند روز قابل انتقال هست؟ [/INST] سلام تا    ساعت </s>',
 '<s>[INST] با سلام...بنده 27 فروردین آخرین پریودیم بوده....27 اردیبهشت دیگر پرید نشدم...الان 7 روز گذشته...4 روز بعد از تاخیر بی بی چک زدم دو خط داشت اما خط پایینی خیلی کمرنگ بود...6 روز بعد از تاخیر آزمایش خون دادم بتام 107 بود میخوام بدونم آیا باردارم یا خیر و اینکه اگر باردارم علت پایین بودن تیتر بتا چیست؟ [/INST] باردار هستيد و يک هفته ديگه سونو بديد </s>',
 '<s>[INST] باسلام\nچهل ساله هستم فرزند سوم غربالگری اول مشکوک به سندروم داون زدن\nگفتن آمنیوسنتز انجام بدم آیا خطر نداره؟ [/INST] امنيوسنتز يک درصد خطر سقط داره اما قطعا بايد انجام بشه </s>',
 '<s>[INST] سلام وقت بخیر من دوهفته پیش هنگام رابطه مویرگ خونی مشاهده کردم به پزشک مراجعه کردم دهانه واژنم آف زده بود آنتی بیوتیک دادن خوب شدم بلا فاصله بعدش پریود شدم به مدت 1 هفته بعد از 2 هفته رابطه جنسیم و شروع کردم پس

In [188]:
te = df_b['question'][0]
te

'سلام. وقت خوش\nمن به ویروس گوارشی مبتلا شدم (شنا میکنم). میخواستم بدونم از شروع علائم مثل تهوع و دلدرد تا چند روز نمیتونم از استخر استفاده کنم و این تا چند روز قابل انتقال هست؟'

In [191]:
te.replace('\n', ' ')

'سلام. وقت خوش من به ویروس گوارشی مبتلا شدم (شنا میکنم). میخواستم بدونم از شروع علائم مثل تهوع و دلدرد تا چند روز نمیتونم از استخر استفاده کنم و این تا چند روز قابل انتقال هست؟'

In [216]:
def clean_t(text):
    t = text
    t2 = t.replace('\n', ' ')
    t3 = t2.replace('\u200c', ' ')
    t4 = re.sub('[a-z]', '', t3)
    return t3.strip()

In [217]:
df['answer'] = df['answer'].apply(clean_t)

In [218]:
df['question'] = df['question'].apply(clean_t)

In [219]:
df['question'][0]

'با سلام من49سالمه سه سال پیش یائسه شدم و الان خونریزی دارم 5روزهست دیروز سونودادم جوابش گفت آندومتروضخامت رحم 9mmهست امروز جمعه بودم نتونستم به دکتری نشان دهم نگرانمواین هم هست که من 20روز بود از کاکوتی استفاده میکردم در چایی هام ودوسه به هم زعفران دم کرده خوردمنشانه بیماری من چی هستممننونم'

In [220]:
df_b = df[50:101]

In [221]:
df_b.reset_index(drop = True, inplace = True)

In [222]:
b = []
for i in range(df_b.shape[0]):
    b.append('<s>[INST] ' + df_b['question'][i] + ' [/INST] ' + df_b['answer'][i] + ' </s>')

In [223]:
b

['<s>[INST] سلام. وقت خوشمن به ویروس گوارشی مبتلا شدم (شنا میکنم). میخواستم بدونم از شروع علائم مثل تهوع و دلدرد تا چند روز نمیتونم از استخر استفاده کنم و این تا چند روز قابل انتقال هست؟ [/INST] سلام تا    ساعت </s>',
 '<s>[INST] با سلام...بنده 27 فروردین آخرین پریودیم بوده....27 اردیبهشت دیگر پرید نشدم...الان 7 روز گذشته...4 روز بعد از تاخیر بی بی چک زدم دو خط داشت اما خط پایینی خیلی کمرنگ بود...6 روز بعد از تاخیر آزمایش خون دادم بتام 107 بود میخوام بدونم آیا باردارم یا خیر و اینکه اگر باردارم علت پایین بودن تیتر بتا چیست؟ [/INST] باردار هستيد و يک هفته ديگه سونو بديد </s>',
 '<s>[INST] باسلامچهل ساله هستم فرزند سوم غربالگری اول مشکوک به سندروم داون زدنگفتن آمنیوسنتز انجام بدم آیا خطر نداره؟ [/INST] امنيوسنتز يک درصد خطر سقط داره اما قطعا بايد انجام بشه </s>',
 '<s>[INST] سلام وقت بخیر من دوهفته پیش هنگام رابطه مویرگ خونی مشاهده کردم به پزشک مراجعه کردم دهانه واژنم آف زده بود آنتی بیوتیک دادن خوب شدم بلا فاصله بعدش پریود شدم به مدت 1 هفته بعد از 2 هفته رابطه جنسیم و شروع کردم پس از از

In [224]:
data = pd.DataFrame({
    'text': b
})

In [225]:
data.head()

,text
0,<s>[INST] سلام. وقت خوشمن به ویروس گوارشی مبتل...
1,<s>[INST] با سلام...بنده 27 فروردین آخرین پریو...
2,<s>[INST] باسلامچهل ساله هستم فرزند سوم غربالگ...
3,<s>[INST] سلام وقت بخیر من دوهفته پیش هنگام را...
4,<s>[INST] بنده 4 ماهه سزارین با بیهوشی عمومی ش...


In [226]:
train_dict = DatasetDict({'train': Dataset.from_pandas(data)})

In [227]:
train_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 51
    })
})

In [228]:
train_data = train_dict['train']

In [229]:
train_data

Dataset({
    features: ['text'],
    num_rows: 51
})

In [230]:
train_data[0]

{'text': '<s>[INST] سلام. وقت خوشمن به ویروس گوارشی مبتلا شدم (شنا میکنم). میخواستم بدونم از شروع علائم مثل تهوع و دلدرد تا چند روز نمیتونم از استخر استفاده کنم و این تا چند روز قابل انتقال هست؟ [/INST] سلام تا    ساعت </s>'}

In [231]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [232]:
logging.set_verbosity_warning()

In [233]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'

In [234]:
new_model = 'llama-2-7b-chuk-test'

In [235]:
output_dir = '/kaggle/working/'

In [236]:
num_train_epochs = 40

In [237]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant = False
)

In [238]:
torch.cuda.empty_cache()

In [239]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [240]:
model.config.use_cache = False

In [241]:
model.config.pretraining_tp = 1

In [242]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [243]:
def text_generation(prompt):
    pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
    result = pipe(f'<s>[INST] {prompt} [/INST]')
    print(result[0]['generated_text'])

In [244]:
text_generation('سلام خوبی؟')

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] سلام خوبی؟ [/INST]  Hello! *smiles* How can I help you today? Is there something you need assistance with or would you like to chat?


In [245]:
text_generation('می توانی فارسی صحبت کنی؟')

Device set to use cuda:0


<s>[INST] می توانی فارسی صحبت کنی؟ [/INST]  Certainly! Here are some common Iranian phrases and words that you may find useful when traveling in Iran:
 sierpand (hello): مرحبا (marhaba) / سلام (salam)
goodbye: خداحافظ


In [246]:
text_generation('برج میلاد در کجا واقع شده است؟ به زبان فارسی جواب بده')

Device set to use cuda:0


<s>[INST] برج میلاد در کجا واقع شده است؟ به زبان فارسی جواب بده [/INST]  The exact location of the birthplace of Jesus Christ is not known with certainty, as there are different accounts and interpretations of the events surrounding his birth. Hinweis: The


In [247]:
torch.cuda.empty_cache()

In [248]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation = True,
        padding = 'max_length',
        max_length = 512
    )

In [249]:
tokenized_dataset = train_data.map(preprocess_function, batched = True)

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [250]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [251]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 1,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [252]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments
)

Truncating train dataset:   0%|          | 0/51 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [253]:
torch.cuda.empty_cache()

In [254]:
print('Starting training...')
trainer.train()
print('Training complete...')

Starting training...


Step,Training Loss
1,4.232500
2,4.107200
3,3.737900
4,3.286900
5,3.469500
6,2.704300
7,2.463100
8,2.136700
9,1.732600
10,1.532000


Training complete...


In [255]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('llama-2-7b-chuk-test/tokenizer_config.json',
 'llama-2-7b-chuk-test/special_tokens_map.json',
 'llama-2-7b-chuk-test/tokenizer.model',
 'llama-2-7b-chuk-test/added_tokens.json')

In [257]:
pr = 'سلام وقت بخیر میخواستم بدونم ال دی رو چجوری باید مصرف کرد 17 سالمه'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 256)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] سلام وقت بخیر میخواستم بدونم ال دی رو چجوری باید مصرف کرد 17 سالمه [/INST] اگر منع مصرف نداشته باشيد از روز اول پريود روزي يکي 


In [ ]:
 '<s>[INST] باسلا و خسته نباشید من 38 سالم هست و تخمدان پلیکیستیک شدید دارم 6 ماه پیش دکتر رفتم و ایشون سه ماه قرص دوفامد تجویز کردن که بعد تموم شدن قرصهادوم فروردین امسال پریود شدم و دوباره 29 فروردین پریود شدم و تقریبا از 15 اردیبهشت سینه هام درد داشت و علائم پریودی داشتم ولی نشدم والان هم که 3 روز از وقتم میگذره پریود نشدم و سینه هام هم خوب شدن و بی بی چکم منفی نشون میده چیکار باید کنم تورو خدا جواب بدین اصلا ممکن هست باردار باشم یا نه غیر ممکنه با تنبلی تخمدان خیلی ممنونم [/INST] در صورتي که تنبلي داشته باشيد بايد هميشه براي منظم شدن پريود قرص بخوريد و نه بصورت دوره اي.تست خون بارداري بديد اگر منفي بود براي پريود به پزشک مراجعه کنيد </s>',


In [265]:
pr = 'با سلام...بنده 27 فروردین آخرین پریودیم بوده....27 اردیبهشت دیگر پرید نشدم...الان 7 روز گذشته...4 روز بعد از تاخیر بی بی چک زدم دو خط داشت اما خط پایینی خیلی کمرنگ بود...6 روز بعد از تاخیر آزمایش خون دادم بتام 107 بود میخوام بدونم آیا باردارم یا خیر و اینکه اگر باردارم علت پایین بودن تیتر بتا چیست؟'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 512)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] با سلام...بنده 27 فروردین آخرین پریودیم بوده....27 اردیبهشت دیگر پرید نشدم...الان 7 روز گذشته...4 روز بعد از تاخیر بی بی چک زدم دو خط داشت اما خط پایینی خیلی کمرنگ بود...6 روز بعد از تاخیر آزمایش خون دادم بتام 107 بود میخوام بدونم آیا باردارم یا خیر و اینکه اگر باردارم علت پایین بودن تیتر بتا چیست؟ [/INST] باردار هستيد و يک هفته ديگه سونو بديد 


In [276]:
pr = 'باسلام و خسته نباشید من 38 سالم هست و تخمدان پلیکیستیک شدید دارم 6 ماه پیش دکتر رفتم و ایشون سه ماه قرص دوفامد تجویز کردن که بعد تموم شدن قرصهادوم فروردین امسال پریود شدم و دوباره 29 فروردین پریود شدم و تقریبا از 15 اردیبهشت سینه هام درد داشت و علائم پریودی داشتم ولی نشدم والان هم که 3 روز از وقتم میگذره پریود نشدم و سینه هام هم خوب شدن و بی بی چکم منفی نشون میده چیکار باید کنم تورو خدا جواب بدین اصلا ممکن هست باردار باشم یا نه غیر ممکنه با تنبلی تخمدان خیلی ممنون'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 700)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] باسلام و خسته نباشید من 38 سالم هست و تخمدان پلیکیستیک شدید دارم 6 ماه پیش دکتر رفتم و ایشون سه ماه قرص دوفامد تجویز کردن که بعد تموم شدن قرصهادوم فروردین امسال پریود شدم و دوباره 29 فروردین پریود شدم و تقریبا از 15 اردیبهشت سینه هام درد داشت و علائم پریودی داشتم ولی نشدم والان هم که 3 روز از وقتم میگذره پریود نشدم و سینه هام هم خوب شدن و بی بی چکم منفی نشون میده چیکار باید کنم تورو خدا جواب بدین اصلا ممکن هست باردار باشم یا نه غیر ممکنه با تنبلی تخمدان خیلی ممنون [/INST] در صورتي که تنبلي داشته باشيد بايد هر هفته از وقتم بعد تموم شدن قرص ها بدين روزا رو با نشون بايد 


In [277]:
pr = 'سلام ما 13 اردیبهشت اقدام به بارداری کردیمالان 17 روز ازش میگذره من میتونم فردا صبح برم آزمایش خون بدم؟؟کمر درد هم دارم'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 256)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] سلام ما 13 اردیبهشت اقدام به بارداری کردیمالان 17 روز ازش میگذره من میتونم فردا صبح برم آزمایش خون بدم؟؟کمر درد هم دارم [/INST] مهم زمان اقدام نيست وقتي تست بديد که تاخير در پريود داشته باشيد 
